# Import

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [3]:
train_df = pd.read_csv("./data/train_df_translate_del.csv")
test_df = pd.read_csv("./data/test_df_translate_del.csv")  
submission = pd.read_csv("./data/sample_submission.csv")  

In [4]:
print(len(test_df))
print(len(submission))

23405
23405


In [5]:
import pandas as pd

# NaN 값을 찾는 함수
def find_nan_values(df, column_name):
    nan_rows = df[df[column_name].isna()]
    return nan_rows

# train_df와 test_df에서 '키워드' 열의 NaN 값 찾기
nan_train = find_nan_values(train_df, '키워드')
nan_test = find_nan_values(test_df, '키워드')

print("Train DataFrame에서 NaN 값이 있는 행들:")
print(nan_train)

print("\nTest DataFrame에서 NaN 값이 있는 행들:")
print(nan_test)

Train DataFrame에서 NaN 값이 있는 행들:
Empty DataFrame
Columns: [ID, 분류, 제목, 키워드]
Index: []

Test DataFrame에서 NaN 값이 있는 행들:
Empty DataFrame
Columns: [ID, 제목, 키워드]
Index: []


## 전처리

영어로만 구성된 단어에 대한 처리 (주소 제거)

In [6]:
import pandas as pd
import re

# 제거할 단어 리스트
remove_words = ['http', 'www.', '.kr', '.net', '.com']

def remove_specific_words(text):
    words = text.split(',')
    filtered_words = [word for word in words if not any(remove_word in word for remove_word in remove_words)]
    return ','.join(filtered_words)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_specific_words)
test_df['키워드'] = test_df['키워드'].apply(remove_specific_words)

In [7]:
print(len(test_df))
print(len(submission))

23405
23405


In [8]:
import pandas as pd
import re

# 영어로만 된 단어를 필터링하는 함수
def is_english(word):
    return bool(re.match('^[a-zA-Z]+$', word))

# 영어로만 된 단어를 추출하는 함수
def extract_english_keywords(keywords):
    words = keywords.split(',')
    english_words = [word for word in words if is_english(word)]
    return english_words

# 모든 영어 단어를 하나의 리스트로 모으기
all_english_words = []
train_df['키워드'].apply(lambda x: all_english_words.extend(extract_english_keywords(x)))

# 리스트를 시리즈로 변환하여 value_counts 호출
english_word_counts = pd.Series(all_english_words).value_counts()

# 상위 10개의 결과 출력
print(english_word_counts.head(10))

AI       3608
SK       3319
GTX      2956
LH       2919
IC       1842
SNS      1644
CCTV     1532
GH       1510
KLPGA    1410
TF       1386
dtype: int64


단어 확인용

In [9]:
# # Pandas 출력 옵션 설정
# pd.set_option('display.max_rows', None)

# english_word_counts.head(20)

단어 종류 총 갯수 확인

In [10]:
len(english_word_counts)

5541

갯수가 2개이하인 단어 제거

In [11]:
import pandas as pd
import re

# 영어로만 된 단어를 필터링하는 함수
def is_english(word):
    return bool(re.match('^[a-zA-Z]+$', word))

# 영어로만 된 단어를 추출하는 함수
def extract_english_keywords(keywords):
    words = keywords.split(',')
    english_words = [word for word in words if is_english(word)]
    return english_words

# 모든 영어 단어를 하나의 리스트로 모으기
train_english_words = []
test_english_words = []

train_df['키워드'].apply(lambda x: train_english_words.extend(extract_english_keywords(x)))
test_df['키워드'].apply(lambda x: test_english_words.extend(extract_english_keywords(x)))

# 리스트를 시리즈로 변환하여 value_counts 호출
train_english_word_counts = pd.Series(train_english_words).value_counts()
test_english_word_counts = pd.Series(test_english_words).value_counts()

# 2개 이하로 등장하는 단어 제거
train_words_to_remove = train_english_word_counts[train_english_word_counts <= 2].index.tolist()
test_words_to_remove = test_english_word_counts[test_english_word_counts <= 2].index.tolist()

# 원본 데이터에서 해당 단어들을 제거하는 함수
def remove_infrequent_words(keywords, words_to_remove):
    words = keywords.split(',')
    filtered_words = [word for word in words if word not in words_to_remove]
    return ','.join(filtered_words)

# 원본 데이터에 반영
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_infrequent_words(x, train_words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_infrequent_words(x, test_words_to_remove))

# 상위 10개의 결과 출력
filtered_train_english_word_counts = train_english_word_counts[train_english_word_counts > 2]
filtered_test_english_word_counts = test_english_word_counts[test_english_word_counts > 2]

print("Train DataFrame 상위 10개의 결과:")
print(filtered_train_english_word_counts.head(10))

print("\nTest DataFrame 상위 10개의 결과:")
print(filtered_test_english_word_counts.head(10))

Train DataFrame 상위 10개의 결과:
AI       3608
SK       3319
GTX      2956
LH       2919
IC       1842
SNS      1644
CCTV     1532
GH       1510
KLPGA    1410
TF       1386
dtype: int64

Test DataFrame 상위 10개의 결과:
AI       1575
SK       1358
GTX      1275
LH       1255
GH        836
IC        776
SNS       717
TF        619
ESG       613
KLPGA     579
dtype: int64


In [12]:
print(len(test_df))
print(len(submission))

23405
23405


단어 갯수 확인용

In [ ]:
filtered_train_english_word_counts.head(20)

제거후 train 영어단어 종류 갯수

In [14]:
len(filtered_train_english_word_counts)

2352

제거후 test 영어단어 종류 갯수

In [15]:
len(filtered_test_english_word_counts)

1452

해당단어 있는지 찾기용

In [16]:
# # 정확히 'City' 단어를 포함하고 있는 행 필터링 / City플랫폼 이런 단어는 찾지 않음
# filtered_df = train_df[train_df['키워드'].str.contains(r'\bCity\b', na=False)]

# # 결과 출력
# len(filtered_df)

74

In [ ]:
# # 'City' 단어를 포함하고 있는 행 필터링 / City플랫폼 이런 단어 포함 찾음
# filtered_df = train_df[train_df['키워드'].str.contains(r'City', na=False)]

# # 결과 출력
# len(filtered_df)

.